In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.1 MB/s eta 0:00:00


In [4]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Get text files from drive folder

In [5]:
import os

In [6]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the folder containing your text files on Google Drive
folder_path = '/content/drive/MyDrive/NLP analysis'

# Define the desired order of labels
label_order = {
    'Book 1 - The Philosopher\'s Stone.txt': 1,
    'Book 2 - The Chamber of Secrets.txt': 2,
    'Book 3 - The Prisoner of Azkaban.txt': 3,
    'Book 4 - The Goblet of Fire.txt': 4,
    'Book 5 - The Order of the Phoenix.txt': 5,
    'Book 6 - The Half Blood Prince.txt': 6,
    'Book 7 - The Deathly Hallows.txt': 7
}

# Prepare the training data by reading the text files
texts = []
labels = []

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path) and file_name.endswith('.txt'):
        with open(file_path, 'r') as f:
            text = f.read()
            texts.append(text)
            label = label_order.get(file_name)
            if label is not None:
                labels.append(label)

                print(f"File: {file_name}, Label: {label}")

Mounted at /content/drive
File: Book 1 - The Philosopher's Stone.txt, Label: 1
File: Book 2 - The Chamber of Secrets.txt, Label: 2
File: Book 3 - The Prisoner of Azkaban.txt, Label: 3
File: Book 4 - The Goblet of Fire.txt, Label: 4
File: Book 5 - The Order of the Phoenix.txt, Label: 5
File: Book 6 - The Half Blood Prince.txt, Label: 6
File: Book 7 - The Deathly Hallows.txt, Label: 7


preprocessing and training the model

In [7]:
# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
num_labels = 3  # Number of sentiment labels (e.g., positive, negative, neutral)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [10]:
# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)  # Assuming three sentiment classifications

# Preprocess and tokenize the text data
def preprocess_text(text):
    return ' '.join(text.strip().split())

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [15]:

# Perform sentiment analysis for each book
for i in range(len(texts)):
    book_title = os.listdir(folder_path)[i]
    book_text = texts[i]
    
    preprocessed_text = preprocess_text(book_text)
    input_ids = tokenizer.encode(preprocessed_text, add_special_tokens=True, truncation=True, max_length=512)
    
    # Convert the input to a PyTorch tensor
    inputs = torch.tensor([input_ids])
    
    # Perform sentiment analysis
    outputs = model(inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()
    
    # Apply softmax to get the label probabilities
    probabilities = F.softmax(logits, dim=1)
    sentiment_scores = probabilities.squeeze().tolist()
    
    # Print the book title, predicted sentiment label, and sentiment scores
    sentiment_labels = ['Negative', 'Neutral', 'Positive']
    print(f"Book: {book_title}")
    print(f"Predicted Sentiment: {sentiment_labels[predicted_label]}")
    print(f"Sentiment Scores: {sentiment_scores}")
    print()

Book: Book 1 - The Philosopher's Stone.txt
Predicted Sentiment: Positive
Sentiment Scores: [0.29867061972618103, 0.31451937556266785, 0.38680997490882874]

Book: Book 2 - The Chamber of Secrets.txt
Predicted Sentiment: Positive
Sentiment Scores: [0.34104543924331665, 0.2478511482477188, 0.41110336780548096]

Book: Book 3 - The Prisoner of Azkaban.txt
Predicted Sentiment: Positive
Sentiment Scores: [0.3013050854206085, 0.29768964648246765, 0.4010053277015686]

Book: Book 4 - The Goblet of Fire.txt
Predicted Sentiment: Positive
Sentiment Scores: [0.3138362467288971, 0.31383588910102844, 0.3723278343677521]

Book: Book 5 - The Order of the Phoenix.txt
Predicted Sentiment: Positive
Sentiment Scores: [0.33884790539741516, 0.21680514514446259, 0.44434696435928345]

Book: Book 6 - The Half Blood Prince.txt
Predicted Sentiment: Positive
Sentiment Scores: [0.32361891865730286, 0.319620817899704, 0.35676029324531555]

Book: Book 7 - The Deathly Hallows.txt
Predicted Sentiment: Positive
Sentiment